# POC-English-disAmb Tests `2016-07-04`
1st test with old Grammar Terster and v.0.5 hierarchical generalization.  
Link Grammar dictionary words renamed `word@1, word@2` ⇒ `word.a, word.b`
    
This is a multi-run version of [http://langlearn.singularitynet.io/data/clustering_2018/html/POC-English-disAmb-2018-07-04.html](http://langlearn.singularitynet.io/data/clustering_2018/html/POC-English-disAmb-2018-07-04.html)  
Each line is calculated 5x, parsing metrics tested 3x for each calculation.  
The detailed calculation table is shared as 'long_table.txt' in data folder  
[http://langlearn.singularitynet.io/data/clustering_2018/POC-English-disAmb-2018-07-04-multi/](http://langlearn.singularitynet.io/data/clustering_2018/POC-English-disAmb-2018-07-04-multi/)  
This notebook is shared as static html via  
[http://langlearn.singularitynet.io/data/clustering_2018/html/POC-English-disAmb-2018-07-04-multi.html](http://langlearn.singularitynet.io/data/clustering_2018/html/POC-English-disAmb-2018-07-04-multi.html)  

## Basic settings

In [1]:
import os, sys, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path: sys.path.append(module_path)
src_path = module_path + '/src'
if os.path.exists(src_path) and src_path not in sys.path: sys.path.append(src_path)
lg_path = '/home/oleg/miniconda3/envs/ull4/lib/python3.6/site-packages/linkgrammar'
if os.path.exists(lg_path) and lg_path not in sys.path: sys.path.append(lg_path)
from src.utl.utl import UTC
from src.utl.read_files import check_dir
from src.utl.turtle import html_table
from src.grammar_learner.poc05 import learn_grammar, params, parse_metrics, run_learn_grammar
prefix = ''
tmpath = module_path + '/tmp/'
check_dir(tmpath, True, 'none')
print(UTC(), ':: module_path =', module_path)
start = time.time()

2018-07-04 14:41:13 UTC :: module_path = /home/oleg/language-learning


## Grammar Learner parameters

In [2]:
corpus = 'POC-English-disAmb'
out_dir = module_path + '/output/'+ corpus + '-' + str(UTC())[:10] + '-multi'
#dataset = ['R=6_Weight=6:R_mst-weight=+1:R', 'R=6_Weight=1_mst-weight=+1:R', 'LG_ANY_all_parses']
kwargs = {
    'left_wall'     :   ''          ,
    'period'        :   False       ,
    'context'       :   2           ,
    'word_space'    :   'discrete'  ,
    'clustering'    :   'group'     ,
    'grammar_rules' :   2           ,
    'verbose'       :   'min'       ,
    'tmpath'        :   tmpath      , 
    'categories_generalization': 'off',
    'rules_generalization': 'off',
    'test_corpus'   : module_path + \
        '/data/POC-English-Amb/poc_english.txt',
    'reference_path': module_path + \
        '/data/POC-English-disAmb/poc-english_ex-parses-gold.txt',
    'template_path' : 'poc-turtle', 
    'linkage_limit' : 1 }

## Test function

In [3]:
def run_test(**kwargs):
    spaces = ''
    if kwargs['context'] == 1: 
        spaces += 'c'
    else: spaces += 'd'
    if kwargs['word_space'] == 'vectors': 
        spaces += 'DRK'
    else: spaces += 'ILE'
    if kwargs['grammar_rules'] == 1: 
        spaces += 'c'
    else: spaces += 'd'
    details = []
    average = []
    for dataset in ['R=6_Weight=6:R_mst-weight=+1:R', 'R=6_Weight=1_mst-weight=+1:R', 'LG_ANY_all_parses']:
        for kwargs['left_wall'] in ['']:
            if kwargs['left_wall'] == 'LEFT-WALL': 
                lw = 'LW'
                kwargs['period'] = True
                dot = ' + '
            else: 
                lw = ' -- '
                dot = ' -- '
                kwargs['period'] = False
            for g in [('',''), ('', 'jaccard'), ('jaccard',''), ('jaccard','jaccard')]:
                if lw == 'LW' and dot == ' + ' and g != ('',''): continue
                elif kwargs['grammar_rules'] == 1 and g[0] != '': continue
                elif kwargs['word_space'] == 'vectors' and g[1] != '': continue
                elif kwargs['context'] == 1 and g[1] != '': continue
                kwargs['rules_generalization'] = g[0]
                kwargs['categories_generalization'] = g[1]
                if g[0] == '' and g[1] == '': gen = 'none'
                elif g[0] == 'jaccard' and g[1] == '': gen = 'rules'
                elif g[0] == '' and g[1] == 'jaccard': gen = 'categories'
                else: gen = 'both'
                ip, oc, og = params(corpus, dataset, module_path, out_dir, **kwargs)
                pa = []
                pq = []
                rules = []
                for j in range(5):
                    r = learn_grammar(ip, oc, og, **kwargs)
                    for i in range(2):
                        a, q, lg_parse_path = parse_metrics(r['grammar_file'], **kwargs)
                        pa.append(a)
                        pq.append(q)                            
                        rules.append(r['grammar_rules'])
                        rulestr = ' ' + str(r['grammar_rules']) + ' '
                        line = [corpus, dataset, lw, dot, gen, spaces, rulestr, \
                            str(a)+'%', str(q)+'%', str(int(round(a*q/100, 0)))+'%']
                        details.append(line)
                paa = int(round(sum(pa)/len(pa), 0))
                pqa = int(round(sum(pq)/len(pq), 0))
                rules_avg = int(round(sum(rules)/len(rules), 0))
                avg = [corpus, dataset, lw, dot, gen, spaces, rules_avg, \
                    str(paa)+'%', str(pqa)+'%', str(int(round(paa*pqa/100, 0)))+'%']
                average.append(avg)
    return average, details

# POC-English-disAmb, 3 datasets
ULL Project Plan ⇒ Parses ⇒ lines 23-28

In [4]:
table = []
long_table = []

## Connectors-DRK-Connectors

In [5]:
%%capture
kwargs['context'] = 1
kwargs['word_space'] = 'vectors'
kwargs['clustering'] = 'kmeans'
kwargs['grammar_rules'] = 1
average21, long21 = run_test(**kwargs)
table.extend(average21)
long_table.extend(long21)

In [6]:
header = ['Corpus','Parsing','LW','"."','Generalization','Space','Rules','PA','PQ','PQA']
display(html_table([header]+average21))

Corpus,Parsing,LW,""".""",Generalization,Space,Rules,PA,PQ,PQA
POC-English-disAmb,R=6_Weight=6:R_mst-weight=+1:R,--,--,none,cDRKc,27,83%,51%,42%
POC-English-disAmb,R=6_Weight=1_mst-weight=+1:R,--,--,none,cDRKc,20,86%,56%,48%
POC-English-disAmb,LG_ANY_all_parses,--,--,none,cDRKc,26,86%,56%,48%


## Connectors-DRK-Disjuncts

In [7]:
%%capture
kwargs['context'] = 1
kwargs['word_space'] = 'vectors'
kwargs['clustering'] = 'kmeans'
kwargs['grammar_rules'] = 2
average22, long22 = run_test(**kwargs)
table.extend(average22)
long_table.extend(long22)

In [8]:
display(html_table([header]+average22))

Corpus,Parsing,LW,""".""",Generalization,Space,Rules,PA,PQ,PQA
POC-English-disAmb,R=6_Weight=6:R_mst-weight=+1:R,--,--,none,cDRKd,28,58%,33%,19%
POC-English-disAmb,R=6_Weight=6:R_mst-weight=+1:R,--,--,rules,cDRKd,26,67%,40%,27%
POC-English-disAmb,R=6_Weight=1_mst-weight=+1:R,--,--,none,cDRKd,20,69%,46%,32%
POC-English-disAmb,R=6_Weight=1_mst-weight=+1:R,--,--,rules,cDRKd,19,78%,49%,38%
POC-English-disAmb,LG_ANY_all_parses,--,--,none,cDRKd,26,70%,43%,30%
POC-English-disAmb,LG_ANY_all_parses,--,--,rules,cDRKd,23,76%,48%,36%


## Disjuncts-DRK-Disjuncts

In [9]:
%%capture
kwargs['context'] = 2
kwargs['word_space'] = 'vectors'
kwargs['clustering'] = 'kmeans'
kwargs['grammar_rules'] = 2
average23, long23 = run_test(**kwargs)
table.extend(average23)
long_table.extend(long23)

In [10]:
display(html_table([header]+average23))

Corpus,Parsing,LW,""".""",Generalization,Space,Rules,PA,PQ,PQA
POC-English-disAmb,R=6_Weight=6:R_mst-weight=+1:R,--,--,none,dDRKd,20,45%,27%,12%
POC-English-disAmb,R=6_Weight=6:R_mst-weight=+1:R,--,--,rules,dDRKd,20,52%,34%,18%
POC-English-disAmb,R=6_Weight=1_mst-weight=+1:R,--,--,none,dDRKd,13,66%,42%,28%
POC-English-disAmb,R=6_Weight=1_mst-weight=+1:R,--,--,rules,dDRKd,12,67%,43%,29%
POC-English-disAmb,LG_ANY_all_parses,--,--,none,dDRKd,16,73%,46%,34%
POC-English-disAmb,LG_ANY_all_parses,--,--,rules,dDRKd,13,75%,48%,36%


## Disjuncts-ILE-Disjuncts

In [11]:
%%capture
kwargs['context'] = 2
kwargs['word_space'] = 'discrete'
kwargs['clustering'] = 'group'
kwargs['grammar_rules'] = 2
average24, long24 = run_test(**kwargs)
table.extend(average24)
long_table.extend(long24)

In [12]:
display(html_table([header]+average24))

Corpus,Parsing,LW,""".""",Generalization,Space,Rules,PA,PQ,PQA
POC-English-disAmb,R=6_Weight=6:R_mst-weight=+1:R,--,--,none,dILEd,39,40%,24%,10%
POC-English-disAmb,R=6_Weight=6:R_mst-weight=+1:R,--,--,categories,dILEd,36,45%,29%,13%
POC-English-disAmb,R=6_Weight=6:R_mst-weight=+1:R,--,--,rules,dILEd,36,45%,29%,13%
POC-English-disAmb,R=6_Weight=6:R_mst-weight=+1:R,--,--,both,dILEd,36,45%,29%,13%
POC-English-disAmb,R=6_Weight=1_mst-weight=+1:R,--,--,none,dILEd,41,37%,26%,10%
POC-English-disAmb,R=6_Weight=1_mst-weight=+1:R,--,--,categories,dILEd,37,37%,27%,10%
POC-English-disAmb,R=6_Weight=1_mst-weight=+1:R,--,--,rules,dILEd,37,37%,27%,10%
POC-English-disAmb,R=6_Weight=1_mst-weight=+1:R,--,--,both,dILEd,37,37%,27%,10%
POC-English-disAmb,LG_ANY_all_parses,--,--,none,dILEd,39,58%,35%,20%
POC-English-disAmb,LG_ANY_all_parses,--,--,categories,dILEd,33,64%,42%,27%


# All tests: each test 5x(parse metrics tested 2x)

In [13]:
display(html_table([header]+long_table))

Corpus,Parsing,LW,""".""",Generalization,Space,Rules,PA,PQ,PQA
POC-English-disAmb,R=6_Weight=6:R_mst-weight=+1:R,--,--,none,cDRKc,26,83%,51%,42%
POC-English-disAmb,R=6_Weight=6:R_mst-weight=+1:R,--,--,none,cDRKc,26,83%,51%,42%
POC-English-disAmb,R=6_Weight=6:R_mst-weight=+1:R,--,--,none,cDRKc,28,82%,51%,42%
POC-English-disAmb,R=6_Weight=6:R_mst-weight=+1:R,--,--,none,cDRKc,28,82%,51%,42%
POC-English-disAmb,R=6_Weight=6:R_mst-weight=+1:R,--,--,none,cDRKc,25,83%,51%,42%
POC-English-disAmb,R=6_Weight=6:R_mst-weight=+1:R,--,--,none,cDRKc,25,83%,51%,42%
POC-English-disAmb,R=6_Weight=6:R_mst-weight=+1:R,--,--,none,cDRKc,26,83%,51%,42%
POC-English-disAmb,R=6_Weight=6:R_mst-weight=+1:R,--,--,none,cDRKc,26,83%,51%,42%
POC-English-disAmb,R=6_Weight=6:R_mst-weight=+1:R,--,--,none,cDRKc,30,83%,51%,42%
POC-English-disAmb,R=6_Weight=6:R_mst-weight=+1:R,--,--,none,cDRKc,30,83%,51%,42%


In [14]:
from src.utl.write_files import list2file
table_str = list2file(table, out_dir+'/short_table.txt')
long_table_str = list2file(long_table, out_dir+'/long_table.txt')

In [15]:
print(UTC(), ':: finished, elapsed', \
      str(int(round((time.time()-start)/60, 0))), \
      'min\nJoint table saved to', out_dir + '/long_table.txt')

2018-07-04 14:43:42 UTC :: finished, elapsed 2 min
Joint table saved to /home/oleg/language-learning/output/POC-English-disAmb-2018-07-04-multi/long_table.txt
